In [ ]:
import requests, bs4, time
import pandas as pd
import os.path
from datetime import datetime

In [ ]:
path = os.getcwd()
parent_folder, current_folder = os.path.split(path)

# Scrape all links to job postings containing the word 'Data'
First searching for all job postings using the word 'Data' on www.cwjobs.de.uk

In [ ]:
def extract_jobtype(soup):
    jobtype=[x.text for x in soup.find_all('li',{'class':'job-type'})]
    return jobtype

def extract_full_desc(soup):
    text=[x.text for x in soup.find_all('div',{'class':'job-description'})]
    return text

def extract_links(soup):
    links =[]
    for div in soup.find_all(name='div', attrs={'class':'job-title'}):
        for a in div.find_all('a'):
            links.append(a['href'])
    return links

def extract_company_from_result(soup): 
    company = []
    for div in soup.find_all(name="li", attrs={"class":"company"}):
        company.append(div.text.strip())
        
    return(company)

def extract_date_from_result(soup): 
    date = []
    for div in soup.find_all('li',{'class':'date-posted'}):
        date.append(div.text.strip())
   
    return(date)

def extract_location_from_result(soup): 
    location = []
    for div in soup.find_all('li', {'class':'location'}):
        for a in div.find('a'):
            location.append(a)
   
    return(location)

def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name="li", attrs={"class":"salary"}):
        try:
            salaries.append(div.text)
        except:
            salaries.append("Nothing_found")
    return(salaries)

def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"job-title"}):
        for a in div.find_all(name="h2"):
            jobs.append(a)
    return(jobs)

In [ ]:
#scraping code:

#decide what search term to use for finding jobs
searchTerm="data"

#create empty data frame with column headers
ads=pd.DataFrame(columns=['company','title','salary','location','date','full_description','jobtype','url'])

# loop for scraping

for i in range(0, 123):
    company = []
    job_title = []
    description = []
    salary = []
    location = []
    date = []
    full_description = []
    text_list = []
    type_list = []
    
    time.sleep(1) #ensuring at least 1 second between page grabs
    url = 'https://www.cwjobs.co.uk/jobs/'+searchTerm+'?s=header&page='+str(i)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.content)
    df = pd.DataFrame(columns=['company','title','salary','location','date','full_description','jobtype','url'])
    df['company'] = extract_company_from_result(soup)
    df['title'] = extract_job_title_from_result(soup)
    df['salary'] = extract_salary_from_result(soup)
    df['location'] = extract_location_from_result(soup)
    df['date'] = extract_date_from_result(soup)
    
    sub_urls=extract_links(soup)
    for j in sub_urls:
        res_sub = requests.get(j)
        soup_sub = bs4.BeautifulSoup(res_sub.content)
        desc = extract_full_desc(soup_sub)
        jobtype = extract_jobtype(soup_sub)
        text_list.append(desc)
        type_list.append(jobtype)
        
    df['full_description'] = text_list
    df['jobtype'] = type_list
    df['url'] = sub_urls

    ads = ads.append(df, ignore_index=True)

today = datetime.now().strftime('%Y_%m_%d_%H_%M')
ads.to_csv(parent_folder+'/data/cwjobs_'+today+'.csv', index=True, sep='\t')

In [ ]:
ads

In [ ]:
#establish how many ads had no salary
ads['salary'].value_counts()